# FBXO11
[Intellectual developmental disorder with dysmorphic facies and behavioral abnormalities (IDDFBA)](https://omim.org/entry/618089) is caused by heterozygous mutation in the [FBXO11 gene](https://omim.org/entry/607871).

F-box proteins, such as FBXO11, contain an approximately 40-amino acid motif, the F box, that binds SKP1

import genophenocorr

In [3]:
import gpsea

In [12]:
import gpsea
from gpsea.analysis import configure_cohort_analysis, CohortAnalysisConfiguration
from gpsea.analysis.predicate import PatientCategories
from gpsea.preprocessing import load_phenopackets
from gpsea.preprocessing import configure_caching_cohort_creator
from gpsea.model import FeatureType, VariantEffect
from gpsea.view import CohortViewable
from gpsea.preprocessing import UniprotProteinMetadataService
from gpsea.model.genome import GRCh38
from gpsea.preprocessing import VVMultiCoordinateService
from gpsea.view import ProteinVisualizable, ProteinVisualizer, ProteinViewable
import hpotk
from IPython.display import display, HTML

store = hpotk.configure_ontology_store()
hpo = store.load_minimal_hpo()
print(f'Loaded HPO v{hpo.version}')
print(f"Using gpsea version {gpsea.__version__}")

Loaded HPO v2024-08-13
Using gpsea version 0.2.1.dev0


In [6]:
from ppktstore.registry import configure_phenopacket_registry
registry = configure_phenopacket_registry()
with registry.open_phenopacket_store(release="0.1.19") as ps:
  phenopackets = list(ps.iter_cohort_phenopackets("FBXO11"))
print(f"Found cohort with {len(phenopackets)} phenopackets for FBXO11")

Found cohort with 20 phenopackets for FBXO11


In [14]:
FBXO11_MANE_transcript = 'NM_001190274.2'
FBXO11_protein_id = 'NP_001177203.1' # F-box only protein 11 isoform 4
cohort_creator = configure_caching_cohort_creator(hpo, timeout=20)
cohort = load_phenopackets(phenopackets=phenopackets, cohort_creator=cohort_creator)

Patients Created: 100%|██████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:42<00:00,  2.12s/it]


In [16]:
cv = CohortViewable(hpo=hpo)
html = cv.process(cohort=cohort, transcript_id=FBXO11_MANE_transcript)
display(HTML(html))

AttributeError: 'tuple' object has no attribute 'list_present_phenotypes'

SyntaxError: invalid syntax (2072620363.py, line 1)